In [14]:
from sklearn import datasets
import matplotlib.pyplot as plt # 画图工具
import numpy as np
import pandas as pd
%matplotlib inline

from sklearn.manifold import TSNE

target = pd.read_csv("wisc_bc_ContinuousVar.csv")
dataset = pd.read_csv("wisc_bc_ContinuousVar.csv")
target['diagnosis'] = target['diagnosis'].map({'M':1,'B':-1})
target = target.diagnosis
dataset = dataset.drop(columns=['id','diagnosis'],axis=1)

X = TSNE(random_state=123,
                   n_components=2,
                   perplexity=4, 
                   learning_rate=150.0,
                   verbose=1,angle=0.1).fit_transform(dataset)

[t-SNE] Computing 13 nearest neighbors...
[t-SNE] Indexed 569 samples in 0.002s...
[t-SNE] Computed neighbors for 569 samples in 0.008s...
[t-SNE] Computed conditional probabilities for sample 569 / 569
[t-SNE] Mean sigma: 10.571699
[t-SNE] KL divergence after 250 iterations with early exaggeration: 66.250420
[t-SNE] KL divergence after 1000 iterations: 0.453708


In [15]:
X

array([[-54.543545 ,  35.99793  ],
       [-60.577003 ,  45.06833  ],
       [-31.241484 ,  40.502773 ],
       ...,
       [  1.4412832, -65.35702  ],
       [-42.797348 ,  50.46403  ],
       [-25.001854 ,  82.4725   ]], dtype=float32)

In [16]:
y = target

In [17]:
y

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19    -1
20    -1
21    -1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
539   -1
540   -1
541   -1
542   -1
543   -1
544   -1
545   -1
546   -1
547   -1
548   -1
549   -1
550   -1
551   -1
552   -1
553   -1
554   -1
555   -1
556   -1
557   -1
558   -1
559   -1
560   -1
561   -1
562    1
563    1
564    1
565    1
566    1
567    1
568   -1
Name: diagnosis, Length: 569, dtype: int64

In [18]:

# X, y = datasets.make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=1, random_state=2)
# for i in range(X.shape[0]):
#     if y[i] == 0:
#         y[i] = -1


class SMO:
    def __init__(self, X, y, C, toler, maxIter):  # samples labels constance tolerate maxIteration
        self.X = X
        self.y = y
        self.C = C
        self.toler = toler
        self.maxIter = maxIter
        self.N = self.X.shape[0]
        self.b = 0
        self.a = np.zeros(self.N)
        self.K = np.zeros((self.N, self.N))
        self.o = 0.5
        for i in range(self.N):
            for j in range(self.N):
                t = 0
                for k in range(self.X.shape[1]):
                    t += (self.X[i][k] - self.X[j][k])**2
                self.K[i][j] = np.exp(-t/(2*self.o**2))

    def select_J(self, i):  # random choose j which is not equal to i.
        j = i
        while j == i:
            j = np.random.randint(0, self.N)
        return j

    def fix_alpha(self, a, H, L):
        if a > H:
            a = H
        if a < L:
            a = L
        return a

    def cal_Ei(self, j):
        t = 0
        for i in range(self.N):
            t += self.a[i]*self.y[i]*self.K[i][j]
        return t + self.b - self.y[j]

    def update(self):
        iter = 0
        while iter < self.maxIter:
            alphaPairsChanged = 0
            for i in range(self.N):
                Ei = self.cal_Ei(i)
                if (Ei < -self.toler and self.a[i] < self.C) or (Ei > self.toler and self.a[i] > 0):
                    j = self.select_J(i)  # choose j != i
                    Ej = self.cal_Ei(j)
                    aiold = self.a[i]
                    ajold = self.a[j]
                    if self.y[i] != self.y[j]:
                        L = max(0, self.a[j] - self.a[i])
                        H = min(self.C, self.C + self.a[j] - self.a[i])
                    else:
                        L = max(0, self.a[j] + self.a[i] - self.C)
                        H = min(self.C, self.a[j] + self.a[i])
                    if L == H:
                        print("L==H")
                        continue
                    eta = self.K[i][i] + self.K[j][j] - 2*self.K[i][j]
                    if eta <= 0 :
                        print('eta <= 0')
                        continue
                    ajnew = ajold + self.y[j]*(Ei - Ej)/eta
                    ajnew = self.fix_alpha(ajnew, H, L)
                    if abs(ajnew - ajold) < 0.00001:
                        print("j not moving enough")
                        continue
                    self.a[j] = self.fix_alpha(ajnew, H, L)
                    self.a[i] = aiold + self.y[i]*self.y[j]*(ajold - ajnew)
                    b1 = -Ei - self.y[i]*self.K[i][i]*(self.a[i] - aiold) - self.y[j]*self.K[i][j]*(ajnew - ajold) + self.b
                    b2 = -Ej - self.y[i]*self.K[i][j]*(self.a[i] - aiold) - self.y[j]*self.K[j][j]*(ajnew - ajold) + self.b
                    if 0 < self.a[i] < self.C:
                        self.b = b1
                    elif 0 < self.a[j] < self.C:
                        self.b = b2
                    else:
                        self.b = (b1 + b2)/2
                    alphaPairsChanged += 1
            if alphaPairsChanged == 0:
                iter += 1
            else:
                iter = 0
        return self.b, self.a

    def score(self, X, y):
        K = np.zeros((X.shape[0], self.N))
        for i in range(X.shape[0]):
            for j in range(self.N):
                t = 0
                for k in range(self.X.shape[1]):
                    t += (self.X[i][k] - self.X[j][k]) ** 2
                K[i][j] = np.exp(-t / (2 * self.o ** 2))
        y_pre = []
        for i in range(X.shape[0]):
            t = 0
            for j in range(self.N):
                t += self.a[j]*self.y[j]*K[i][j]
            y_pre.append(t + self.b)
        s = 0
        for i in range(X.shape[0]):
            if y_pre[i] < 0 and y[i] == -1:
                s += 1
            elif y_pre[i] > 0 and y[i] == 1:
                s += 1
        return s/X.shape[0]


L = SMO(X, y, 1, 0.001, 100)
b, a = L.update()
score = L.score(X, y)
print('b = {}'.format(b))
print('a = {}'.format(a))
print('the accuracy of this SVM is {}'.format(score))



L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
L==H
L==H
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
j not

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough


j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving en

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving en

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving en

...

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not

j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not

j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not movi

j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough


j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
b = -0.31743146492837343
a = [1.         0.72899808 1.         1.         1.         0.92884163
 1.         0.73534812 0.90830133 0.8666466  1.         0.90019328
 1.         1.         1.         1.         0.75475381 0.92221544
 0.82807421 0.56464038 0.62152542 0.65685473 1.        